In [6]:
import sys
import os
from pathlib import Path
# This appends the directory one level up (the root of your project) to the sys.path.
# Modify the path depending on the location of modules you want to import.
sys.path.append(os.path.abspath('../../'))

from config.config_managers import DashboardConfigManager
from dataManager import DataManager
from dash import Dash
import pandas as pd
import plotly.express as px
from abc import ABC, abstractmethod
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
CONFIG_PATH = Path("/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/dashboard-config.yaml")
config_manager = DashboardConfigManager(CONFIG_PATH)
dev_config = config_manager.development_config    

app = Dash(__name__, suppress_callback_exceptions=True)

app_config = config_manager.app_config
server = app.server  # Flask server instance for caching
variants_data = None

data_manager = DataManager(config_manager, server)
dash_data = data_manager.load_data()

2025-03-22 15:37:33 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/validations/My Drive
2025-03-22 15:37:33 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2025-03-22 15:37:33 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02


  0%|          | 0/18 [00:00<?, ?it/s]

2025-03-22 15:37:38 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/validations/My Drive
2025-03-22 15:37:38 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2025-03-22 15:37:38 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/conll2003_bert


  0%|          | 0/18 [00:00<?, ?it/s]

In [8]:
ar = dash_data['ANERCorp_CamelLab_arabertv02']
en = dash_data['conll2003_bert']

In [12]:
df = en.analysis_data
df.columns


Index(['Sentence Ids', 'Token Positions', 'Words', 'Tokens', 'Word Pieces',
       'Core Tokens', 'True Labels', 'Token Selector Id', 'Pred Labels',
       'Agreements', 'X', 'Y', 'Labels', 'Losses', 'Token Ids', 'Global Id',
       'True Silhouette', 'Pred Silhouette', 'K=3', 'Boundary Clusters', 'K=4',
       'Entity Clusters', 'K=9', 'Token Clusters', 'Consistency Count',
       'Inconsistency Count', 'Total Train Occurrences', 'Local Token Entropy',
       'Token Max Entropy', 'Dataset Token Entropy', 'Local Word Entropy',
       'Word Max Entropy', 'Dataset Word Entropy', 'Tokenization Rate',
       'Error Type', 'O Confidence', 'B-PER Confidence', 'I-PER Confidence',
       'B-ORG Confidence', 'I-ORG Confidence', 'B-LOC Confidence',
       'I-LOC Confidence', 'B-MISC Confidence', 'I-MISC Confidence',
       'Prediction Entropy', 'Prediction Max Entropy', 'Token Confidence',
       'Variability', 'Pre X', 'Pre Y', 'Strict True Entities',
       'Strict Pred Entities', 'True Entiti

In [13]:
df[(df['Labels']!=-100)]["Confusion Components"].value_counts()

Confusion Components
TN    38088
TP     7555
FP      669
FN      123
Name: count, dtype: int64

In [16]:
1563+589+1523+1560+237+161+777+1145

7555

In [14]:
df['Error Type'].value_counts()

Error Type
No Errors            69575
Type                   382
Inclusion              235
Exclusion              123
Boundary                36
Type and Boundary       16
Name: count, dtype: int64

In [15]:
# Function to classify each token
def classify_ner(true_label, predicted_label):
    if predicted_label == true_label and predicted_label != "O":
        return "TP"
    elif predicted_label != true_label and predicted_label != "O":
        return "FP"
    elif predicted_label != true_label and true_label != "O" and predicted_label == "O":
        return "FN"
    else:
        return "TN"  # Optional if you want True Negatives

# Apply classification
df["Confusion Components"] = df.apply(lambda row: classify_ner(row["True Labels"], row["Pred Labels"]), axis=1)

In [10]:
df[~df["Confusion Components"].isin(['TP', 'TN'])][["Confusion Components", "Error Type", 'True Labels', 'Pred Labels']]

,Confusion Components,Error Type,True Labels,Pred Labels
1,FN,Exclusion,B-LOC,O
50,FN,Exclusion,B-PER,O
163,FP,Inclusion,O,B-LOC
379,FP,Boundary,I-PER,B-PER
415,FN,Exclusion,I-ORG,O
...,...,...,...,...
29503,FN,Exclusion,I-ORG,O
29504,FN,Exclusion,I-ORG,O
29505,FN,Exclusion,I-ORG,O
29539,FP,Boundary,I-PER,B-PER


In [15]:
df[(df['Labels']!=-100)]["Confusion Components"].value_counts()

Confusion Components
TN    21472
TP     2813
FP      395
FN      311
Name: count, dtype: int64

In [17]:
633+159+355+754+68+68+199+577

2813